<a href="https://colab.research.google.com/github/Imsachin010/UDR-Fused-CNN-Models/blob/main/Densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet201

# Define the paths to your dataset directories
benign_dir = r'/content/drive/MyDrive/DATABASES/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549AB/100X'
malignant_dir = r'/content/drive/MyDrive/DATABASES/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-10926/100X'

# Load and preprocess the data
image_size = (32, 32)

image_files = []
labels = []

# Load benign images
for filename in os.listdir(benign_dir):
    if filename.endswith('.png'):
        image_path = os.path.join(benign_dir, filename)
        image_files.append(image_path)
        labels.append(0)  # 0 represents benign

# Load malignant images
for filename in os.listdir(malignant_dir):
    if filename.endswith('.png'):
        image_path = os.path.join(malignant_dir, filename)
        image_files.append(image_path)
        labels.append(1)  # 1 represents malignant

# Split the data into a training (70%) and testing (30%) set
X_train, X_test, y_train, y_test = train_test_split(image_files, labels, test_size=0.3, random_state=42)

# Create a function to load and preprocess images
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, image_size)
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img

# Create TensorFlow datasets for training and testing
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(lambda x, y: (preprocess_image(x), y))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(lambda x, y: (preprocess_image(x), y))
test_dataset = test_dataset.batch(batch_size)

# Define the model architecture
num_classes = 2
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


74836368/74836368 [==============================] - 0s 0us/step


In [ ]:
# model layers
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 38, 38, 3)            0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 16, 16, 64)           9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1/conv[0][0]']      

In [ ]:
#flowchart of model
from tensorflow.keras.utils import plot_model
plot_model(model)

In [ ]:
#Train the model
epochs = 10
model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)


Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
# To use Support Vector Machines (SVM) with a pre-trained DenseNet201 model for binary classification on the same dataset, you can extract features from the pre-trained model and then train an SVM classifier on those features. Here's how you can do it:

# python
# Copy code
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet201
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report

# Rest of your code for loading and preprocessing data

# Define the model architecture and extract features
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
base_model.trainable = False

# Create a new model that outputs features
x = base_model.output
x = GlobalAveragePooling2D()(x)
feature_model = Model(inputs=base_model.input, outputs=x)

# Extract features for training and testing data
X_train_features = feature_model.predict(train_dataset.map(lambda x, y: x))
X_test_features = feature_model.predict(test_dataset.map(lambda x, y: x))

# Train an SVM classifier
svm_classifier = svm.SVC(kernel='linear')  # You can experiment with different kernels
svm_classifier.fit(X_train_features, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test_features)

# Evaluate the SVM classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)

In [ ]:
#testing
loss, acc = model.evaluate(test_dataset)
print("loss: ",loss)
print("accuracy: ",acc)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Preprocess the test dataset
X_test_preprocessed = [preprocess_image(image_path) for image_path in X_test]

# Convert the list of preprocessed images to a numpy array
X_test_preprocessed = np.array(X_test_preprocessed)

# Predict probabilities
y_pred_prob = model.predict(X_test_preprocessed)

# Convert probabilities to binary predictions using a threshold of 0.5
y_pred = (y_pred_prob > 0.5).astype(int)

print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

X_test_preprocessed = np.array(X_test_preprocessed)  # Convert X_test_preprocessed to a numpy array

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cbar=False, cmap=plt.cm.Blues, vmax=X_test_preprocessed.shape[0] // 16)
